In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision.transforms import transforms
import torchvision.datasets as datasets

In [2]:
class NN(nn.Module):
    def __init__(self, input_size, num_classes):
      super(NN, self).__init__()
      self.fc1 = nn.Linear(input_size, 50)
      self.fc2 = nn.Linear(50, num_classes)


    def forward(self, x):
       x = F.relu(self.fc1(x))
       x = self.fc2(x)
       return x

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
input_size = 784
num_classes = 10
batch_size = 64
num_epochs = 1
learning_rate = 0.001

In [5]:
train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

100%|██████████| 9912422/9912422 [00:00<00:00, 100033952.60it/s]


Extracting dataset/MNIST/raw/train-images-idx3-ubyte.gz to dataset/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 68709979.48it/s]

Extracting dataset/MNIST/raw/train-labels-idx1-ubyte.gz to dataset/MNIST/raw




100%|██████████| 1648877/1648877 [00:00<00:00, 28683671.69it/s]


Extracting dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to dataset/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 14941591.19it/s]

Extracting dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to dataset/MNIST/raw



In [6]:
model = NN(input_size, num_classes=num_classes).to(device)

In [7]:
criterion = nn.CrossEntropyLoss()

In [8]:
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [9]:
for epoch in range(num_epochs):
    for batc_idx, (data, targets) in enumerate(train_loader):
       data = data.to(device=device)
       targets = targets.to(device=device)
       data = data.reshape(data.shape[0], -1)
       scores = model(data)
       loss = criterion(scores, targets)
       optimizer.zero_grad()
       loss.backward()
       optimizer.step()

In [10]:
def check_accuracy(loader, model):
   if loader.dataset.train:
      print("checking accuracy on training data")
   else:
      print("checking accuracy on testing data")

   num_correct = 0
   num_samples = 0
   model.eval()

   with torch.no_grad():
      for x, y in loader:
         x = x.to(device=device)
         y = y.to(device=device)
         x = x.reshape(x.shape[0], -1)

         scores = model(x)
         _, predictions = scores.max(1)
         num_correct += (predictions == y).sum()
         num_samples += predictions.size(0)

      print(f"got {num_correct}/{num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}")

      model.train()

check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

checking accuracy on training data
got 55959/60000 with accuracy 93.27
checking accuracy on testing data
got 9299/10000 with accuracy 92.99
